In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from openpoints.utils import EasyConfig

In [2]:
import argparse
import yaml
import os
import logging
import numpy as np
import csv
import wandb
import glob
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import distributed as dist, multiprocessing as mp
from torch.utils.tensorboard import SummaryWriter
from torch_scatter import scatter
from torch_geometric.data import Data, Batch
from openpoints.utils import (
    set_random_seed,
    save_checkpoint,
    load_checkpoint,
    resume_checkpoint,
    setup_logger_dist,
    cal_model_parm_nums,
    Wandb,
    generate_exp_directory,
    resume_exp_directory,
    EasyConfig,
    dist_utils,
    find_free_port,
    load_checkpoint_inv,
)
from openpoints.utils import AverageMeter, ConfusionMatrix, get_mious
from openpoints.dataset import (
    build_dataloader_from_cfg,
    get_features_by_keys,
    get_class_weights,
    get_class_alpha
)
from openpoints.dataset.data_util import voxelize
from openpoints.dataset.semantic_kitti.semantickitti import (
    load_label_kitti,
    load_pc_kitti,
    remap_lut_read,
    remap_lut_write,
    get_semantickitti_file_list,
)
from openpoints.transforms import build_transforms_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.loss import build_criterion_from_cfg
from openpoints.dataset import build_dataset_from_cfg
# from openpoints.models import build_model_from_cfg

import warnings

In [3]:
cfg = EasyConfig()
cfg.load("../cfgs/forinstancev2/default.yaml", recursive=True)

In [4]:
cfg.dataset.common.data_root = "../data/FORInstanceV2"

In [5]:
cfg.dataset.common.variable = True
cfg.dataset.val.presample=True
cfg.dataset.train.presample=True

In [6]:
dataset = build_dataset_from_cfg(cfg.dataset.common, default_args=cfg.dataset.train)


Tiling LASDataset train split:   0%|          | 0/48 [00:00<?, ?it/s]

../data/FORInstanceV2/raw/train/BlueCat_RN_merged_trees_train_2cm.laz


Tiling LASDataset train split:   2%|▏         | 1/48 [08:24<6:35:24, 504.79s/it]

../data/FORInstanceV2/raw/train/CULS_CULS_plot_1_annotated_train.laz


Tiling LASDataset train split:   4%|▍         | 2/48 [08:30<2:41:42, 210.93s/it]

...

Tiling LASDataset train split:  96%|█████████▌| 46/48 [11:29<00:14,  7.44s/it]

../data/FORInstanceV2/raw/train/TUWIEN_TUWIEN_train_train.laz


Tiling LASDataset train split:  98%|█████████▊| 47/48 [11:53<00:12, 12.22s/it]

../data/FORInstanceV2/raw/train/Yuchen_2023_dls_merged_230209_panoptic_train.laz


Loading LASDataset train split: 100%|██████████| 48903/48903 [06:50<00:00, 119.16it/s]


In [12]:
cfg.dataset.val.presample = False

dataset = build_dataset_from_cfg(cfg.dataset.common, default_args=cfg.dataset.val)

In [21]:
dataset2.transform

In [25]:
val_transform = build_transforms_from_cfg("val", cfg.datatransforms)

In [26]:
val_transform

In [29]:
cfg.dataset.val.presample = True

dataset2 = build_dataset_from_cfg(cfg.dataset.common, default_args=cfg.dataset.val)

In [34]:
dataset2[0]['pos'] == dataset[0]['pos']

array([[ True,  True,  True],
       [ True,  True, False],
       [ True,  True,  True],
       ...,
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]], shape=(9000, 3))

In [36]:
dataset.data_list[0]

'../data/FORInstanceV2/tiled/val/BlueCat_RN_merged_trees_val_2cm_0.0_0.0_0.0_1.npy'

In [20]:
arr - arr.min(0)

array([[0.36, 1.44, 2.32, 0.  ],
       [0.34, 1.44, 2.32, 0.  ],
       [0.34, 1.42, 2.32, 0.  ],
       ...,
       [0.38, 1.52, 2.3 , 0.  ],
       [0.38, 1.52, 2.32, 0.  ],
       [0.02, 1.2 , 2.04, 0.  ]], shape=(14478, 4))

In [16]:
dataset2[0]

{'pos': array([[0.63998413, 1.3000183 , 0.47999573],
        [0.63998413, 1.3000183 , 0.55999756],
        [0.63998413, 1.3000183 , 0.5200043 ],
        ...,
        [0.65997314, 1.0200195 , 0.2400055 ],
        [0.65997314, 1.        , 0.19999695],
        [0.65997314, 1.        , 0.16000366]],
       shape=(9000, 3), dtype=float32),
 'x': None,
 'y': array([1, 1, 1, ..., 1, 1, 1], shape=(9000,)),
 'heights': tensor([[0.4800],
         [0.5600],
         [0.5200],
         ...,
         [0.2400],
         [0.2000],
         [0.1600]])}

In [14]:
dataset.data

AttributeError: 'FORInstanceV2' object has no attribute 'data'

In [8]:
dataset = build_dataset_from_cfg(cfg.dataset.common, default_args=cfg.dataset.test)

Tiling LASDataset test split:   0%|          | 0/29 [00:00<?, ?it/s]

../data/FORInstanceV2/raw/test/BlueCat_RN_merged_trees_test_2cm.laz


Tiling LASDataset test split:   3%|▎         | 1/29 [00:55<25:51, 55.42s/it]

../data/FORInstanceV2/raw/test/CULS_CULS_plot_2_annotated_test.laz


Tiling LASDataset test split:   7%|▋         | 2/29 [01:03<12:24, 27.58s/it]

...

Tiling LASDataset test split:  93%|█████████▎| 27/29 [02:37<00:11,  5.89s/it]

../data/FORInstanceV2/raw/test/TUWIEN_TUWIEN_test_test.laz


Tiling LASDataset test split:  97%|█████████▋| 28/29 [02:42<00:05,  5.77s/it]

../data/FORInstanceV2/raw/test/Yuchen_2023_dls_merged_230209_panoptic_test.laz


Tiling LASDataset test split: 100%|██████████| 29/29 [02:44<00:00,  5.66s/it]
